# This code is for HOS using multilingual Embeddings for three Dravidian CodeMix languages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Packages to be installed

In [ ]:
%pip install -U sentence-transformers

## Import the required packages

In [ ]:
# packages
import pandas as pd 
from collections import Counter
from sentence_transformers import SentenceTransformer 
import numpy as np
import sklearn
from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    accuracy_score, 
    mean_squared_error, 
    mean_absolute_error,
    classification_report,
    confusion_matrix
)

### Read the data

In [5]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reduced dataset/tiny/tiny train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reduced dataset/tiny/tiny test.csv",header=None)

### Sperate the train and test senetnecs and labels to a list

In [ ]:
train_sent_m=[]
train_label_m=[]
test_sent_m = []
test_label_m = []
print(train.columns)
print(test.columns)
for i in train['0']:
    train_sent_m.append(i.split('\t')[0])
    train_label_m.append(i.split('\t')[0])

for i in test[0]:
    test_sent_m.append(i.split('\t')[0])
    test_label_m.append(i.split('\t')[0])

In [ ]:
print(f"Train sentences: {len(train_sent_m)}, Train labels: {len(train_label_m)}")
print(f"Test sentences: {len(test_sent_m)}, Test labels: {len(test_label_m)}")


### Label Encoding

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_labels_encoded = le.fit_transform(train_label_m)
dev_labels_encoded = le.fit_transform(test_label_m)

### Get Embeddings

In [ ]:
trans_model = SentenceTransformer('bert-base-multilingual-cased')
# here other multilingual embeddings can be loaded

In [13]:
train_sentence_embeddings = trans_model.encode(train_sent_m)
dev_sentence_embeddings = trans_model.encode(test_sent_m)

### Weight calculation

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Correct usage
class_weights = compute_class_weight(
    class_weight='balanced',  # Strategy
    classes=np.unique(train_label_m),  # Unique class labels
    y=train_label_m  # Training labels
)

print("Class Weights:", class_weights)

### Classification

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Ensure LabelEncoder is fit on all unique labels across train and test sets
all_labels = list(set(train_label_m) | set(test_label_m))  # Union of train and test labels
le = LabelEncoder()
le.fit(all_labels)

# Encode train and test labels
train_labels_encoded = le.transform(train_label_m)
dev_labels_encoded = le.transform(test_label_m)

# Dynamically compute class weights for all classes in the training set
class_weight_values = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(train_labels_encoded), 
    y=train_labels_encoded
)
class_weight = {cls: weight for cls, weight in zip(np.unique(train_labels_encoded), class_weight_values)}

# Train Logistic Regression model
model = LogisticRegression(class_weight=class_weight, max_iter=10000)
model.fit(train_sentence_embeddings, train_labels_encoded)

# Make predictions
predicted = model.predict(dev_sentence_embeddings)

# Evaluate metrics
print("Eval Scores:")
accuracy = accuracy_score(dev_labels_encoded, predicted)
recall = recall_score(dev_labels_encoded, predicted, average="macro", zero_division=1)
precision = precision_score(dev_labels_encoded, predicted, average="macro", zero_division=1)
f1 = f1_score(dev_labels_encoded, predicted, average="macro", zero_division=1)

print("Macro Metrics:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

# Generate classification report
unique_labels = sorted(set(dev_labels_encoded))  # Ensure we match actual labels present in the dataset
target_names = le.inverse_transform(unique_labels)  # Map encoded labels back to original labels

print("\nClassification Report:")
print(classification_report(dev_labels_encoded, predicted, labels=unique_labels, target_names=target_names, zero_division=1))

# Save predictions
predictions = list(le.inverse_transform(predicted))  # Decode predictions back to original labels
classified_df = pd.DataFrame({'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('Logistic_regression.csv', index=False)
print("Predictions saved to 'Logistic_regression.csv'")


Eval Scores:
Macro Metrics:
Accuracy: 0.000
Precision: 0.690
Recall: 0.310
F1 Score: 0.000

Classification Report:
                                                                                             precision    recall  f1-score   support

                     2.10- joju ചേട്ടന്റെ ആ ഇരുത്തം കണ്ടിട്ട് കിരീടം ടച്ച്‌ തോന്നിയവർ ഉണ്ടോ       1.00      0.00      0.00       1.0
                                                            Any Kerala vijay fans like hite       1.00      0.00      0.00       1.0
                                                                                    Column1       1.00      0.00      0.00       1.0
           Engane aada uvve 32K dislikes okke vanne? nammude naatil athrem mandanmaar undo        1.00      0.00      0.00       1.0
                                                   Enta joju etta ningalithe ende bhavichaa       1.00      0.00      0.00       1.0
                                                 Fefka ee padam release cheyyan samadhicho?       1.00      0.00      0.00       1.0
    Ninakku eni Malayalam intrustriyil polikkan pattumo boss se Ikka no dilouge only action       1.00      0.00      0.00       1.0
                                                          Oru pandi comedy pandum pole unnd       1.00      0.00      0.00       1.0
                                                         Pwoli ee vishu ikkante kayile urap       1.00      0.00      0.00       1.0
                       Ravile thane views likes ethra ayyi enn nokan Vanavar adi like evide       1.00      0.00      0.00       1.0
                    അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലും ചീറ്റലും ഒക്കെ ഇണ്ടാവും       1.00      0.00      0.00       1.0
                                                                                  അത് സത്യം       1.00      0.00      0.00       1.0
                                അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ  Suraj ആണ് നടൻ ന്ന് തോന്നുന്നു       1.00      0.00      0.00       1.0
                    അപ്പൊ ജോഷി സാർ ഇവിടെയൊക്കെ  തന്നെയുണ്ടല്ലേ ... Thanks ഈ തിരിച്ചു വരവിന്       1.00      0.00      0.00       1.0
                                                           ഈ ഓണം കഴിഞ്ഞിട്ട് അടുത്ത ഓണം ആയി       1.00      0.00      0.00       1.0
                          എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo videoorderil ചെയ്തപോലെ       1.00      0.00      0.00       1.0
                                      നമ്മ കുമ്പളങ്ങിക്കാർ ലൈക്‌ അടി  ഇവിടെ  കാണട്ടെ മാസ്സ്       1.00      0.00      0.00       1.0
                                       പടം കാണാത്തവരുണ്ടങ്കിൽ ഇന്ന് തന്നെ പോയി കണ്ടോ മക്കളെ       1.00      0.00      0.00       1.0
മഞ്ചു വാരിയര്‍ ഇത് ഞെരിക്കും... ഒപ്പം റോഷന്‍ ആണ്ട്ര്യൂസ് അവതരിപ്പിക്കുന്ന വില്ലന്‍ വേഷവും..       1.00      0.00      0.00       1.0
                                            മൂത്തോൻ🤗 നിവിൻ പോളി ഫാൻസ് അടിക്കു മക്കളെ ലൈക്ക്       1.00      0.00      0.00       1.0

                                                                                  micro avg       1.00      0.00      0.00      20.0
                                                                                  macro avg       1.00      0.00      0.00      20.0
                                                                               weighted avg       1.00      0.00      0.00      20.0

Predictions saved to 'Logistic_regression.csv'

### Confusion Matrix

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

# Assuming 'expected' and 'predicted' are your true and predicted labels
cfm = confusion_matrix(expected, predicted)

# Get the unique classes from the actual and predicted labels
classes = np.unique(np.concatenate((expected, predicted)))

# Convert confusion matrix to a DataFrame for better visualization
df_cfm = pd.DataFrame(cfm, index=classes, columns=classes)

# Plot confusion matrix using seaborn heatmap
plt.figure(figsize=(7, 5))
cfm_plot = sn.heatmap(df_cfm, annot=True, fmt='g', cmap='Blues')

# Save the plot as a PNG file
cfm_plot.figure.savefig("cfm_LR.png")

# Show the plot
plt.show()


### Naive Bayse

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Ensure LabelEncoder is fit on all unique labels across train and test sets
all_labels = list(set(train_label_m) | set(test_label_m))  # Union of train and test labels
le = LabelEncoder()
le.fit(all_labels)

# Encode train and test labels
train_labels_encoded = le.transform(train_label_m)
dev_labels_encoded = le.transform(test_label_m)

# Train Naive Bayes model
model = GaussianNB()
model.fit(train_sentence_embeddings, train_labels_encoded)

# Make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)

# Evaluate metrics
print("Eval Scores:")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted, average="macro", zero_division=1)
precision = precision_score(expected, predicted, average="macro", zero_division=1)
f1 = f1_score(expected, predicted, average="macro", zero_division=1)

print("Macro Metrics:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

# Generate classification report
unique_labels = sorted(set(expected))  # Match actual labels in the dataset
target_names = le.inverse_transform(unique_labels)  # Map encoded labels back to original labels

print("\nClassification Report:")
print(classification_report(expected, predicted, labels=unique_labels, target_names=target_names, zero_division=1))

# Save predictions
predictions = list(le.inverse_transform(predicted))  # Decode predictions back to original labels
classified_df = pd.DataFrame({'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('Naive_Bayes.csv', index=False)
print("Predictions saved to 'Naive_Bayes.csv'")


Output

Eval Scores:
Macro Metrics:
Accuracy: 0.000
Precision: 0.690
Recall: 0.310
F1 Score: 0.000

Classification Report:
                                                                                             precision    recall  f1-score   support

                     2.10- joju ചേട്ടന്റെ ആ ഇരുത്തം കണ്ടിട്ട് കിരീടം ടച്ച്‌ തോന്നിയവർ ഉണ്ടോ       1.00      0.00      0.00       1.0
                                                            Any Kerala vijay fans like hite       1.00      0.00      0.00       1.0
                                                                                    Column1       1.00      0.00      0.00       1.0
           Engane aada uvve 32K dislikes okke vanne? nammude naatil athrem mandanmaar undo        1.00      0.00      0.00       1.0
                                                   Enta joju etta ningalithe ende bhavichaa       1.00      0.00      0.00       1.0
                                                 Fefka ee padam release cheyyan samadhicho?       1.00      0.00      0.00       1.0
    Ninakku eni Malayalam intrustriyil polikkan pattumo boss se Ikka no dilouge only action       1.00      0.00      0.00       1.0
                                                          Oru pandi comedy pandum pole unnd       1.00      0.00      0.00       1.0
                                                         Pwoli ee vishu ikkante kayile urap       1.00      0.00      0.00       1.0
                       Ravile thane views likes ethra ayyi enn nokan Vanavar adi like evide       1.00      0.00      0.00       1.0
                    അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലും ചീറ്റലും ഒക്കെ ഇണ്ടാവും       1.00      0.00      0.00       1.0
                                                                                  അത് സത്യം       1.00      0.00      0.00       1.0
                                അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ  Suraj ആണ് നടൻ ന്ന് തോന്നുന്നു       1.00      0.00      0.00       1.0
                    അപ്പൊ ജോഷി സാർ ഇവിടെയൊക്കെ  തന്നെയുണ്ടല്ലേ ... Thanks ഈ തിരിച്ചു വരവിന്       1.00      0.00      0.00       1.0
                                                           ഈ ഓണം കഴിഞ്ഞിട്ട് അടുത്ത ഓണം ആയി       1.00      0.00      0.00       1.0
                          എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo videoorderil ചെയ്തപോലെ       1.00      0.00      0.00       1.0
                                      നമ്മ കുമ്പളങ്ങിക്കാർ ലൈക്‌ അടി  ഇവിടെ  കാണട്ടെ മാസ്സ്       1.00      0.00      0.00       1.0
                                       പടം കാണാത്തവരുണ്ടങ്കിൽ ഇന്ന് തന്നെ പോയി കണ്ടോ മക്കളെ       1.00      0.00      0.00       1.0
മഞ്ചു വാരിയര്‍ ഇത് ഞെരിക്കും... ഒപ്പം റോഷന്‍ ആണ്ട്ര്യൂസ് അവതരിപ്പിക്കുന്ന വില്ലന്‍ വേഷവും..       1.00      0.00      0.00       1.0
                                            മൂത്തോൻ🤗 നിവിൻ പോളി ഫാൻസ് അടിക്കു മക്കളെ ലൈക്ക്       1.00      0.00      0.00       1.0

                                                                                  micro avg       1.00      0.00      0.00      20.0
                                                                                  macro avg       1.00      0.00      0.00      20.0
                                                                               weighted avg       1.00      0.00      0.00      20.0

Predictions saved to 'Naive_Bayes.csv'

## Confusion matrix (CM)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Generate the confusion matrix
cfm = confusion_matrix(expected, predicted, labels=np.arange(len(le.classes_)))

# Dynamically get class labels using LabelEncoder
class_labels = le.classes_  # Original labels from the encoder

# Create a DataFrame for the heatmap
df_cfm = pd.DataFrame(cfm, index=class_labels, columns=class_labels)

# Plot confusion matrix using Seaborn
plt.figure(figsize=(10, 7))
cfm_plot = sn.heatmap(df_cfm, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix - Naive Bayes")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.tight_layout()
plt.savefig("cfm_NB.png")  # Save as PNG file
plt.show()

print("Confusion matrix saved as 'cfm_NB.png'")


output:
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3354 (\N{MALAYALAM LETTER CA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Matplotlib currently does not support Malayalam natively.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3399 (\N{MALAYALAM VOWEL SIGN EE}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3359 (\N{MALAYALAM LETTER TTA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3405 (\N{MALAYALAM SIGN VIRAMA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3368 (\N{MALAYALAM LETTER NA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3377 (\N{MALAYALAM LETTER RRA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3398 (\N{MALAYALAM VOWEL SIGN E}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3334 (\N{MALAYALAM LETTER AA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3335 (\N{MALAYALAM LETTER I}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3376 (\N{MALAYALAM LETTER RA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3393 (\N{MALAYALAM VOWEL SIGN U}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3364 (\N{MALAYALAM LETTER TA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3330 (\N{MALAYALAM SIGN ANUSVARA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3349 (\N{MALAYALAM LETTER KA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3363 (\N{MALAYALAM LETTER NNA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3391 (\N{MALAYALAM VOWEL SIGN I}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3392 (\N{MALAYALAM VOWEL SIGN II}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3403 (\N{MALAYALAM VOWEL SIGN OO}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3375 (\N{MALAYALAM LETTER YA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3381 (\N{MALAYALAM LETTER VA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3452 (\N{MALAYALAM LETTER CHILLU RR}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3337 (\N{MALAYALAM LETTER U}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3333 (\N{MALAYALAM LETTER A}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3385 (\N{MALAYALAM LETTER HA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3390 (\N{MALAYALAM VOWEL SIGN AA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3384 (\N{MALAYALAM LETTER SA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3351 (\N{MALAYALAM LETTER GA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3356 (\N{MALAYALAM LETTER JA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3372 (\N{MALAYALAM LETTER BA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3370 (\N{MALAYALAM LETTER PA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3402 (\N{MALAYALAM VOWEL SIGN O}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3378 (\N{MALAYALAM LETTER LA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3346 (\N{MALAYALAM LETTER O}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3374 (\N{MALAYALAM LETTER MA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3451 (\N{MALAYALAM LETTER CHILLU N}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3383 (\N{MALAYALAM LETTER SSA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3336 (\N{MALAYALAM LETTER II}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3358 (\N{MALAYALAM LETTER NYA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3382 (\N{MALAYALAM LETTER SHA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3347 (\N{MALAYALAM LETTER OO}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3343 (\N{MALAYALAM LETTER EE}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3379 (\N{MALAYALAM LETTER LLA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3342 (\N{MALAYALAM LETTER E}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3400 (\N{MALAYALAM VOWEL SIGN AI}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3380 (\N{MALAYALAM LETTER LLLA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3338 (\N{MALAYALAM LETTER UU}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3361 (\N{MALAYALAM LETTER DDA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3454 (\N{MALAYALAM LETTER CHILLU LL}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3353 (\N{MALAYALAM LETTER NGA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3453 (\N{MALAYALAM LETTER CHILLU L}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3366 (\N{MALAYALAM LETTER DA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3373 (\N{MALAYALAM LETTER BHA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3394 (\N{MALAYALAM VOWEL SIGN UU}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 129303 (\N{HUGGING FACE}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 3371 (\N{MALAYALAM LETTER PHA}) missing from current font.
  fig.canvas.draw()
<ipython-input-29-a446c17dbf18>:22: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3354 (\N{MALAYALAM LETTER CA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Matplotlib currently does not support Malayalam natively.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3399 (\N{MALAYALAM VOWEL SIGN EE}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3359 (\N{MALAYALAM LETTER TTA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3405 (\N{MALAYALAM SIGN VIRAMA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3368 (\N{MALAYALAM LETTER NA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3377 (\N{MALAYALAM LETTER RRA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3398 (\N{MALAYALAM VOWEL SIGN E}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3334 (\N{MALAYALAM LETTER AA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3335 (\N{MALAYALAM LETTER I}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3376 (\N{MALAYALAM LETTER RA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3393 (\N{MALAYALAM VOWEL SIGN U}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3364 (\N{MALAYALAM LETTER TA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3330 (\N{MALAYALAM SIGN ANUSVARA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3349 (\N{MALAYALAM LETTER KA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3363 (\N{MALAYALAM LETTER NNA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3391 (\N{MALAYALAM VOWEL SIGN I}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3392 (\N{MALAYALAM VOWEL SIGN II}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3403 (\N{MALAYALAM VOWEL SIGN OO}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3375 (\N{MALAYALAM LETTER YA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3381 (\N{MALAYALAM LETTER VA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3452 (\N{MALAYALAM LETTER CHILLU RR}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3337 (\N{MALAYALAM LETTER U}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3333 (\N{MALAYALAM LETTER A}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3385 (\N{MALAYALAM LETTER HA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3390 (\N{MALAYALAM VOWEL SIGN AA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3384 (\N{MALAYALAM LETTER SA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3351 (\N{MALAYALAM LETTER GA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3356 (\N{MALAYALAM LETTER JA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3372 (\N{MALAYALAM LETTER BA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3370 (\N{MALAYALAM LETTER PA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3402 (\N{MALAYALAM VOWEL SIGN O}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3378 (\N{MALAYALAM LETTER LA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3346 (\N{MALAYALAM LETTER O}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3374 (\N{MALAYALAM LETTER MA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3451 (\N{MALAYALAM LETTER CHILLU N}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3383 (\N{MALAYALAM LETTER SSA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3336 (\N{MALAYALAM LETTER II}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3358 (\N{MALAYALAM LETTER NYA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3382 (\N{MALAYALAM LETTER SHA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3347 (\N{MALAYALAM LETTER OO}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3343 (\N{MALAYALAM LETTER EE}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3379 (\N{MALAYALAM LETTER LLA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3342 (\N{MALAYALAM LETTER E}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3400 (\N{MALAYALAM VOWEL SIGN AI}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3380 (\N{MALAYALAM LETTER LLLA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3338 (\N{MALAYALAM LETTER UU}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3361 (\N{MALAYALAM LETTER DDA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3454 (\N{MALAYALAM LETTER CHILLU LL}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3353 (\N{MALAYALAM LETTER NGA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3453 (\N{MALAYALAM LETTER CHILLU L}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3366 (\N{MALAYALAM LETTER DA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3373 (\N{MALAYALAM LETTER BHA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3394 (\N{MALAYALAM VOWEL SIGN UU}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 129303 (\N{HUGGING FACE}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
<ipython-input-29-a446c17dbf18>:23: UserWarning: Glyph 3371 (\N{MALAYALAM LETTER PHA}) missing from current font.
  plt.savefig("cfm_NB.png")  # Save as PNG file
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3354 (\N{MALAYALAM LETTER CA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Matplotlib currently does not support Malayalam natively.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3399 (\N{MALAYALAM VOWEL SIGN EE}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3359 (\N{MALAYALAM LETTER TTA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3405 (\N{MALAYALAM SIGN VIRAMA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3368 (\N{MALAYALAM LETTER NA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3377 (\N{MALAYALAM LETTER RRA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3398 (\N{MALAYALAM VOWEL SIGN E}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3334 (\N{MALAYALAM LETTER AA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3335 (\N{MALAYALAM LETTER I}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3376 (\N{MALAYALAM LETTER RA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3393 (\N{MALAYALAM VOWEL SIGN U}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3364 (\N{MALAYALAM LETTER TA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3330 (\N{MALAYALAM SIGN ANUSVARA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3349 (\N{MALAYALAM LETTER KA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3363 (\N{MALAYALAM LETTER NNA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3391 (\N{MALAYALAM VOWEL SIGN I}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3392 (\N{MALAYALAM VOWEL SIGN II}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3403 (\N{MALAYALAM VOWEL SIGN OO}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3375 (\N{MALAYALAM LETTER YA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3381 (\N{MALAYALAM LETTER VA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3452 (\N{MALAYALAM LETTER CHILLU RR}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3337 (\N{MALAYALAM LETTER U}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3333 (\N{MALAYALAM LETTER A}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3385 (\N{MALAYALAM LETTER HA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3390 (\N{MALAYALAM VOWEL SIGN AA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3384 (\N{MALAYALAM LETTER SA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3351 (\N{MALAYALAM LETTER GA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3356 (\N{MALAYALAM LETTER JA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3372 (\N{MALAYALAM LETTER BA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3370 (\N{MALAYALAM LETTER PA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3402 (\N{MALAYALAM VOWEL SIGN O}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3378 (\N{MALAYALAM LETTER LA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3346 (\N{MALAYALAM LETTER O}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3374 (\N{MALAYALAM LETTER MA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3451 (\N{MALAYALAM LETTER CHILLU N}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3383 (\N{MALAYALAM LETTER SSA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3336 (\N{MALAYALAM LETTER II}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3358 (\N{MALAYALAM LETTER NYA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3382 (\N{MALAYALAM LETTER SHA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3347 (\N{MALAYALAM LETTER OO}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3343 (\N{MALAYALAM LETTER EE}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3379 (\N{MALAYALAM LETTER LLA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3342 (\N{MALAYALAM LETTER E}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3400 (\N{MALAYALAM VOWEL SIGN AI}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3380 (\N{MALAYALAM LETTER LLLA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3338 (\N{MALAYALAM LETTER UU}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3361 (\N{MALAYALAM LETTER DDA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3454 (\N{MALAYALAM LETTER CHILLU LL}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3353 (\N{MALAYALAM LETTER NGA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3453 (\N{MALAYALAM LETTER CHILLU L}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3366 (\N{MALAYALAM LETTER DA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3373 (\N{MALAYALAM LETTER BHA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3394 (\N{MALAYALAM VOWEL SIGN UU}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 129303 (\N{HUGGING FACE}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 3371 (\N{MALAYALAM LETTER PHA}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)

Confusion matrix saved as 'cfm_NB.png'

## Random Forest (RF)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
import numpy as np
import pandas as pd

# Dynamically compute class weights
classes = np.unique(train_labels_encoded)  # Get unique class labels
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=classes, 
    y=train_labels_encoded
)
class_weight_dict = dict(zip(classes, class_weights))

# Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, class_weight=class_weight_dict, random_state=42)
model.fit(train_sentence_embeddings, train_labels_encoded)

# Make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)

# Evaluate metrics
print("Evaluation Scores:")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted, average="macro", zero_division=0)
precision = precision_score(expected, predicted, average="macro", zero_division=0)
f1 = f1_score(expected, predicted, average="macro", zero_division=0)

print("Macro Metrics:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

# Generate classification report
# Get unique classes present in the data
unique_classes_in_data = np.unique(np.concatenate((expected, predicted)))
filtered_target_names = [le.classes_[i] for i in unique_classes_in_data]

print("\nClassification Report:")
print(classification_report(expected, predicted, target_names=filtered_target_names, zero_division=0))

# Save predictions
predictions = le.inverse_transform(predicted)  # Decode predictions to original labels
classified_df = pd.DataFrame({'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('RF.csv', index=False)
print("Predictions saved to 'RF.csv'")


Output:
Evaluation Scores:
Macro Metrics:
Accuracy: 0.000
Precision: 0.000
Recall: 0.000
F1 Score: 0.000

Classification Report:
                                                                                                                                                           precision    recall  f1-score   support

                                                                                   2.10- joju ചേട്ടന്റെ ആ ഇരുത്തം കണ്ടിട്ട് കിരീടം ടച്ച്‌ തോന്നിയവർ ഉണ്ടോ       0.00      0.00      0.00       1.0
                                                                                                                          Any Kerala vijay fans like hite       0.00      0.00      0.00       1.0
                                                                                                                                                  Column1       0.00      0.00      0.00       1.0
                                                                         Engane aada uvve 32K dislikes okke vanne? nammude naatil athrem mandanmaar undo        0.00      0.00      0.00       1.0
                                                                                                                 Enta joju etta ningalithe ende bhavichaa       0.00      0.00      0.00       1.0
                                                                                                               Fefka ee padam release cheyyan samadhicho?       0.00      0.00      0.00       1.0
                                                                                                 I am waiting for kappan Moganla and Surya\tnot-malayalam       0.00      0.00      0.00       0.0
                                                                                                                 J A K E S.   B EJ O Y !!!\tNot_offensive       0.00      0.00      0.00       0.0
                                                                  Ninakku eni Malayalam intrustriyil polikkan pattumo boss se Ikka no dilouge only action       0.00      0.00      0.00       1.0
                                                                                                                        Oru pandi comedy pandum pole unnd       0.00      0.00      0.00       1.0
                                                                                  Oru ratchasan feel kittitu ullalo. Bgm athra adipoliyano\tNot_offensive       0.00      0.00      0.00       0.0
                                                                                                                       Pwoli ee vishu ikkante kayile urap       0.00      0.00      0.00       1.0
                                                                                     Ravile thane views likes ethra ayyi enn nokan Vanavar adi like evide       0.00      0.00      0.00       1.0
                         eda rithwik nine njan theriparjilaloo ennitt ninte samkkaram kanichuvale f fff .valarthu dhosham nint ppp\tOffensive_Untargetede       0.00      0.00      0.00       0.0
                                                                                  അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലും ചീറ്റലും ഒക്കെ ഇണ്ടാവും       0.00      0.00      0.00       1.0
                                                                                                                                                അത് സത്യം       0.00      0.00      0.00       1.0
                                                                                              അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ  Suraj ആണ് നടൻ ന്ന് തോന്നുന്നു       0.00      0.00      0.00       1.0
                                                                                  അപ്പൊ ജോഷി സാർ ഇവിടെയൊക്കെ  തന്നെയുണ്ടല്ലേ ... Thanks ഈ തിരിച്ചു വരവിന്       0.00      0.00      0.00       1.0
ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്ട ഞാൻ Royal Mech ആടാ  ആരണ്ട ആരണ്ട മീശ പിരിക്കുന്ന ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്ട ഞാൻ royal Mech ആടാ\tNot_offensive       0.00      0.00      0.00       0.0
                                                                                                                         ഈ ഓണം കഴിഞ്ഞിട്ട് അടുത്ത ഓണം ആയി       0.00      0.00      0.00       1.0
                                                                                        എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo videoorderil ചെയ്തപോലെ       0.00      0.00      0.00       1.0
                                                              ജിമിട്ടൊളികൾ ഇപ്പോളെ unlike അടിച്ചു തുടങ്ങി  അസുരൻ loading..\tOffensive_target_insult_Group       0.00      0.00      0.00       0.0
                                                                                                    നമ്മ കുമ്പളങ്ങിക്കാർ ലൈക്‌ അടി  ഇവിടെ  കാണട്ടെ മാസ്സ്       0.00      0.00      0.00       1.0
                                                                                                     പടം കാണാത്തവരുണ്ടങ്കിൽ ഇന്ന് തന്നെ പോയി കണ്ടോ മക്കളെ       0.00      0.00      0.00       1.0
                                                              മഞ്ചു വാരിയര്‍ ഇത് ഞെരിക്കും... ഒപ്പം റോഷന്‍ ആണ്ട്ര്യൂസ് അവതരിപ്പിക്കുന്ന വില്ലന്‍ വേഷവും..       0.00      0.00      0.00       1.0
                                                                                                          മൂത്തോൻ🤗 നിവിൻ പോളി ഫാൻസ് അടിക്കു മക്കളെ ലൈക്ക്       0.00      0.00      0.00       1.0
ഷൈലോക്ക് ന്റെ നല്ല ടീസർ ആയിട്ട് പോലും ട്രോളി നടന്ന ലാലേട്ടൻ ഫാൻസിന് കിട്ടിയൊരു നല്ലൊരു തിരിച്ചടി തന്നെ ആയിരിന്നു ബിഗ് ബ്രദർ ന്റെ ട്രെയ്‌ലർ\tNot_offensive       0.00      0.00      0.00       0.0

                                                                                                                                                 accuracy                           0.00      20.0
                                                                                                                                                macro avg       0.00      0.00      0.00      20.0
                                                                                                                                             weighted avg       0.00      0.00      0.00      20.0

Predictions saved to 'RF.csv'

## Confusion Matrix

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
cfm = confusion_matrix(expected, predicted)

# Dynamically get class labels
classes = np.unique(expected)  # Ensure the classes match the true labels

# Create a DataFrame for the heatmap
df_cfm = pd.DataFrame(cfm, index=classes, columns=classes)

# Plot confusion matrix using Seaborn
plt.figure(figsize=(10, 7))
cfm_plot = sn.heatmap(df_cfm, annot=True, fmt='g', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title("Confusion Matrix - Random Forest")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.tight_layout()

# Save the confusion matrix as a PNG file
plt.savefig("cfm_RF.png")
plt.show()

print("Confusion matrix saved as 'cfm_RF.png'")


error2:
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-35-c6e7bf3bca52> in <cell line: 14>()
     12 
     13 # Create a DataFrame for the heatmap
---> 14 df_cfm = pd.DataFrame(cfm, index=classes, columns=classes)
     15 
     16 # Plot confusion matrix using Seaborn

2 frames
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/construction.py in _check_values_indices_shape_match(values, index, columns)
    418         passed = values.shape
    419         implied = (len(index), len(columns))
--> 420         raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")
    421 
    422 

ValueError: Shape of passed values is (27, 27), indices imply (20, 20)


## SVM RBF

In [ ]:
# SVM rbf
from sklearn.svm import SVC
from sklearn import svm
class_weight = {0:0.22607331, 1:23.54117647 ,2:13.69505703 ,3:17.07014218 ,4:2.484}
model = svm.SVC(kernel='rbf',C = 1000, class_weight =class_weight)
model = model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('SVM_RBF.csv')
print("prediction saved")


## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_SVM_rbf.png")

## SVM Poly

In [ ]:
# SVM poly
from sklearn.svm import SVC
from sklearn import svm
model = svm.SVC(kernel='poly',C = 1000)
model = model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")


print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('SVM_poly_labse.csv')
print("prediction saved")

#Save model
import pickle
# Save the trained model as a pickle string.
pkl_filename = "poly_labse.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)
file.close()

## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot =True,  fmt='g')
cfm_plot.figure.savefig("cfm_SVM_poly_labse.png")

## SVM Linear

In [ ]:
# SVM Linear
from sklearn.svm import SVC
from sklearn import svm
model = svm.SVC(kernel='linear',C = 10)
model = model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('SVM_linear_labse.csv')
print("prediction saved")

#Save model
import pickle
# Save the trained model as a pickle string.
pkl_filename = "linear_labse.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)
file.close()

## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot =True,  fmt='g')
cfm_plot.figure.savefig("cfm_SVM_linear_labse.png")

## Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100)
model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)

print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('Adaboost.csv')
print("prediction saved")

## Confusion matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_adaboost.png")

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(weights = 'distance')
model.fit(train_sentence_embeddings, train_labels_encoded)


# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('KNN.csv')
print("prediction saved")


## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_KNN.png")

## Decision Tree

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
class_weight = {0:0.22607331, 1:23.54117647 ,2:13.69505703 ,3:17.07014218 ,4:2.484}
model = DecisionTreeClassifier(class_weight=class_weight)
model.fit(train_sentence_embeddings, train_labels_encoded)


# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('DT.csv')
print("prediction saved")
